In [6]:
%pip install tensorflow -q
%pip install pandas -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from tensorflow.keras import models, layers, activations,\
optimizers, utils, losses, initializers, metrics, callbacks

In [19]:
epochs = 1000
batch_size = 64
patience = 32
learning_rate = 1e-3
model_path = 'checkpoints/model.keras'
train_path = 'train'
exists = os.path.exists(model_path)
items = len(os.listdir(train_path))
classes = 1 if items < 3 else items
print(classes)

17


In [22]:
# Carrega modelo se já existir um checkpoint, caso contrário, o cria.
model = models.load_model(model_path) \
if exists \
else models.Sequential([
  layers.Resizing(64, 64),
  layers.Rescaling(1.0/255),
  layers.RandomRotation((-0.1, 0.1)),
  layers.RandomFlip(),
  layers.BatchNormalization(),
  
  layers.Conv2D(64, (5, 5),
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  # layers.MaxPooling2D((3, 3)),
  
  layers.Conv2D(16, (3, 3),
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  layers.MaxPooling2D((3, 3)),
  
  layers.Flatten(),
  layers.Dropout(0.5),
  layers.Dense(256,
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  
  layers.Dense(16,
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  
  layers.Dense(16,
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  
  layers.Dense(16,
    activation = 'relu',
    kernel_initializer = initializers.RandomUniform()
  ),
  
  layers.Dense(classes,
    activation = 'sigmoid',
    kernel_initializer = initializers.RandomUniform()
  )
])

if exists:
  model.summary()
else:
  model.compile(
    optimizer = optimizers.Adam(
      learning_rate = learning_rate
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ 'accuracy' ]
  )
  
train = utils.image_dataset_from_directory(
  train_path,
  validation_split= 0.2,
  subset= "training",
  seed= 123,
  shuffle= True,
  image_size= (224, 224),
  batch_size= batch_size
)

test = utils.image_dataset_from_directory(
  train_path,
  validation_split= 0.2,
  subset= "validation",
  seed= 123,
  shuffle= True,
  image_size= (224, 224),
  batch_size= batch_size
)

Found 1800 files belonging to 17 classes.
Using 1440 files for training.
Found 1800 files belonging to 17 classes.
Using 360 files for validation.


In [23]:
model.fit(train,
  epochs = epochs,
  validation_data = test,
  callbacks= [
    callbacks.EarlyStopping(
      monitor = 'val_loss',
      patience = patience,
      verbose = 1
    ),
    callbacks.ModelCheckpoint(
      filepath = model_path,
      save_weights_only = False,
      monitor = 'loss',
      mode = 'min',
      save_best_only = True
    ),
    callbacks.LearningRateScheduler(
      lambda epoch, lr: 1e-3 * 10 ** -(epoch / 64)
    )
  ]
)

Epoch 1/1000


23/23 [==============================] - 4s 119ms/step - loss: 2.8322 - accuracy: 0.0604 - val_loss: 2.8272 - val_accuracy: 0.0528 - lr: 0.0010
Epoch 2/1000
23/23 [==============================] - 3s 114ms/step - loss: 2.7917 - accuracy: 0.1056 - val_loss: 2.8153 - val_accuracy: 0.1167 - lr: 9.6466e-04
Epoch 3/1000
23/23 [==============================] - 3s 110ms/step - loss: 2.7275 - accuracy: 0.1278 - val_loss: 2.8203 - val_accuracy: 0.1222 - lr: 9.3057e-04
Epoch 4/1000
23/23 [==============================] - 3s 109ms/step - loss: 2.6809 - accuracy: 0.1396 - val_loss: 2.7635 - val_accuracy: 0.1222 - lr: 8.9769e-04
Epoch 5/1000
23/23 [==============================] - 3s 109ms/step - loss: 2.6373 - accuracy: 0.1576 - val_loss: 2.7343 - val_accuracy: 0.1333 - lr: 8.6596e-04
Epoch 6/1000
23/23 [==============================] - 3s 111ms/step - loss: 2.5805 - accuracy: 0.1465 - val_loss: 2.6562 - val_accuracy: 0.1417 - lr: 8.3536e-04
Epoch 7/1000
23/23 [==============================]

KeyboardInterrupt: 